In [1]:
from google.genai import types
from google import genai
import os
import time
import pandas as pd
import random
import yfinance as yf
from utilities import gemini
import csv
import numpy as np
from scipy.stats import uniform, beta
nation_ending={'UK':'.L','US':''}

In [6]:
a=yf.Ticker('MSFT')

In [7]:
a.info

{'address1': 'One Microsoft Way',
 'city': 'Redmond',
 'state': 'WA',
 'zip': '98052-6399',
 'country': 'United States',
 'phone': '425 882 8080',
 'website': 'https://www.microsoft.com',
 'industry': 'Software - Infrastructure',
 'industryKey': 'software-infrastructure',
 'industryDisp': 'Software - Infrastructure',
 'sector': 'Technology',
 'sectorKey': 'technology',
 'sectorDisp': 'Technology',
 'longBusinessSummary': 'Microsoft Corporation develops and supports software, services, devices and solutions worldwide. The Productivity and Business Processes segment offers office, exchange, SharePoint, Microsoft Teams, office 365 Security and Compliance, Microsoft viva, and Microsoft 365 copilot; and office consumer services, such as Microsoft 365 consumer subscriptions, Office licensed on-premises, and other office services. This segment also provides LinkedIn; and dynamics business solutions, including Dynamics 365, a set of intelligent, cloud-based applications across ERP, CRM, power 

In [13]:
gamma_param(1,5)

(0.04, 25.0)

In [8]:

parsing_functions = {'Sector':lambda x:x.get('sector'),
                      'Industry':lambda x:x.get('industry'),
    'Company_Name': lambda obj: obj.get('longName'),
    'Current_Price': lambda obj: obj.get('currentPrice'),
    'Market_Cap': lambda obj: obj.get('marketCap'),
    'Enterprise_Value': lambda obj: obj.get('enterpriseValue'),
    'PE_Ratio_Trailing': lambda obj: obj.get('trailingPE'),
    'PE_Ratio_Forward': lambda obj: obj.get('forwardPE'),
    'Dividend_Yield': lambda obj: obj.get('dividendYield'),
    'Dividend_Rate': lambda obj: obj.get('dividendRate'),
    'Beta': lambda obj: obj.get('beta'),
    'Total_Revenue': lambda obj: obj.get('totalRevenue'),
    'Revenue_Growth': lambda obj: obj.get('revenueGrowth'),
    'Earnings_Growth': lambda obj: obj.get('earningsGrowth'),
    'Profit_Margins': lambda obj: obj.get('profitMargins'),
    'Gross_Margins': lambda obj: obj.get('grossMargins'),
    'Operating_Margins': lambda obj: obj.get('operatingMargins'),
    'Return_On_Assets': lambda obj: obj.get('returnOnAssets'),
    'Return_On_Equity': lambda obj: obj.get('returnOnEquity'),
    'Debt_To_Equity': lambda obj: obj.get('debtToEquity'),
    'Free_Cashflow': lambda obj: obj.get('freeCashflow'),
    'Operating_Cashflow': lambda obj: obj.get('operatingCashflow'),
    'Total_Cash': lambda obj: obj.get('totalCash'),
    'Total_Debt': lambda obj: obj.get('totalDebt'),
    'Recommendation_Key': lambda obj: obj.get('recommendationKey'),
    'Number_Of_Analyst_Opinions': lambda obj: obj.get('numberOfAnalystOpinions'),
    'Target_High_Price': lambda obj: obj.get('targetHighPrice'),
    'Target_Low_Price': lambda obj: obj.get('targetLowPrice'),
    'Target_Mean_Price': lambda obj: obj.get('targetMeanPrice'),
    'Target_Median_Price': lambda obj: obj.get('targetMedianPrice')
}


In [9]:
client = genai.Client(api_key=os.getenv("GOOGLE_API_KEY"))
def summary(full_name,nation,client):
    question='Please summarise the business of '+nation+' company '+full_name+'. Return one sentence within 500 characters.'
    response = client.models.generate_content(
    model="gemini-2.0-flash", contents=question,config=types.GenerateContentConfig(
        tools=[types.Tool(
            google_search=types.GoogleSearchRetrieval
        )]
    )
)
    return response.text


In [3]:
annotated_list=pd.read_csv('data/sp500_annotation.csv',index_col=0)


In [6]:
annotated_list.copy(deep=True).iloc[374:,]


,Exchange,Shortname,Longname,Sector,Industry,Currentprice,Marketcap,Ebitda,Revenuegrowth,City,State,Country,Fulltimeemployees,Longbusinesssummary,Weight
Symbol,,,,,,,,,,,,,,,
FSLR,NMS,"First Solar, Inc.","First Solar, Inc.",Technology,Solar,182.39,19526307840,1.716966e+09,0.108,Tempe,AZ,United States,6700.0,"First Solar, Inc., a solar technology company,...",0.000351
BF-B,NYQ,Brown Forman Inc,Brown-Forman Corporation,Consumer Defensive,Beverages - Wineries & Distilleries,41.28,19369400320,1.198000e+09,-0.084,Louisville,KY,United States,5700.0,"Brown-Forman Corporation, together with its su...",0.000348
LDOS,NYQ,"Leidos Holdings, Inc.","Leidos Holdings, Inc.",Technology,Information Technology Services,144.81,19322431488,2.057000e+09,0.069,Reston,VA,United States,47000.0,"Leidos Holdings, Inc., together with its subsi...",0.000348
CFG,NYQ,"Citizens Financial Group, Inc.","Citizens Financial Group, Inc.",Financial Services,Banks - Regional,43.45,19148457984,NaN,-0.061,Providence,RI,United States,17329.0,"Citizens Financial Group, Inc. operates as the...",0.000345
LH,NYQ,Labcorp Holdings Inc.,Labcorp Holdings Inc.,Healthcare,Diagnostics & Research,228.61,19120779264,1.673500e+09,0.074,Burlington,NC,United States,67000.0,Labcorp Holdings Inc. provides laboratory serv...,0.000344
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
CZR,NMS,"Caesars Entertainment, Inc.","Caesars Entertainment, Inc.",Consumer Cyclical,Resorts & Casinos,32.82,6973593600,3.668000e+09,-0.040,Reno,NV,United States,51000.0,"Caesars Entertainment, Inc. operates as a gami...",0.000125
BWA,NYQ,BorgWarner Inc.,BorgWarner Inc.,Consumer Cyclical,Auto Parts,31.88,6972155904,1.882000e+09,-0.048,Auburn Hills,MI,United States,39900.0,"BorgWarner Inc., together with its subsidiarie...",0.000125
QRVO,NMS,"Qorvo, Inc.","Qorvo, Inc.",Technology,Semiconductors,70.85,6697217024,6.731300e+08,-0.052,Greensboro,NC,United States,8700.0,"Qorvo, Inc. engages in development and commerc...",0.000120


In [ ]:

def stock_list_annotation(annotated_list, client, nation, output_file,to_add=['Company Name','Summary', 'Market Cap'],sleep_params=(1,10)):
    """Get company data with real-time CSV output"""
    def get_market_cap_formatted(market_cap):
        """Convert market cap to readable format"""
        if not market_cap:
            return 'N/A'
        billion = 1_000_000_000
        if market_cap >= billion:
            return f"${round(market_cap/billion, 1)}B"
        else:
            return f"${round(market_cap/1_000_000, 1)}M"
    #output_file="data/"+output_file_name+'_annotation.csv'
    fieldnames = ['Ticker']+to_add+list(annotated_list.columns)
    file_exists = os.path.exists(output_file)
    
    with open(output_file, 'a', newline='') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        if not file_exists:
            writer.writeheader()

        for ticker,info in annotated_list.iterrows():
            try:
                stock = yf.Ticker(ticker+nation_ending[nation])
                data = stock.history(period='1d', interval='1m')
                #record=info.copy(deep=True)
                
                record = {
                    'Ticker': ticker
                }
                for k in to_add:
                    if k in parsing_functions:
                        record[k]=parsing_functions[k](stock.info)
                if 'Summary' in to_add:
                    record['Summary']=summary(record['Company Name'],nation,client)
                print(record['Summary'])
                record.update(info.to_dict())
                writer.writerow(record)  # Immediate write
                csvfile.flush()  # Force OS-level write

            except Exception as e:
                print(f"Error {ticker}: {str(e)}")
                writer.writerow({'Ticker': ticker})  # Partial write

            sleep_time=np.random.gamma(*sleep_params)
            print(sleep_time)
            time.sleep(sleep_time)

In [8]:
stock_list_annotation(annotated_list.iloc[374:,], client,'US', 'data/sp500_annotation_1.csv')

First Solar, Inc. is an American solar technology company that designs, manufactures, and markets photovoltaic solar modules.

11.656417048960437
Brown-Forman manufactures, bottles, and markets a range of alcoholic beverages, including whiskies, tequila, vodka, and wine.

18.079567392438456
Leidos Holdings, Inc. provides technology and engineering services and solutions, including data analytics, cybersecurity, and enterprise IT, to various industries.

4.223072953171488
Citizens Financial Group, headquartered in Providence, RI, offers retail and commercial banking products/services to individuals, small businesses, and institutions.

17.348856214199774
Labcorp provides clinical laboratory services and end-to-end drug development support, offering a range of diagnostic technologies and testing services.

36.543242700006964
VeriSign is a global provider of internet infrastructure and domain name registry services, ensuring the security, stability, and resiliency of the Domain Name Syste

Align Technology designs, manufactures, and markets orthodontics, restorative, and aesthetic dentistry products, most notably the Invisalign system of clear aligners and iTero intraoral scanners.

14.063597662641916
Dollar Tree, Inc. operates discount variety stores like Dollar Tree and Family Dollar, offering merchandise at fixed prices.

55.73319598471916
Expeditors International is a service-based logistics company that provides global supply chain solutions, including airfreight, ocean freight, and customs brokerage, without owning the transportation assets.

4.485453435510785
Everest Group, Ltd. is a global provider of reinsurance and insurance solutions, offering property, casualty, and specialty coverage.

13.22176997916312
Moderna is a biotechnology company that develops messenger RNA (mRNA) therapeutics and vaccines for various diseases.

23.041911240020173
Alliant Energy is a public utility holding company that generates and distributes electricity and natural gas in Iowa and

Allegion plc is a global provider of security products and solutions for homes, businesses, schools, and institutions.

10.953659756070008
Bunge Global SA is a global agribusiness and food company connecting farmers to consumers to deliver essential food, feed and fuel to the world.

37.80199137954981
Assurant is a global provider of risk management solutions, offering specialty insurance products in the auto, lifestyle, and housing protection sectors.

31.917428411777372
Interpublic Group (IPG) is a global advertising and marketing services company specializing in consumer advertising, digital marketing, communications, media buying, and public relations.

16.31265521522425
Franklin Resources is a global investment management company operating as Franklin Templeton, providing investment solutions and related services to various clients worldwide.

26.78793802889289
Eastman Chemical is a global specialty materials company producing a range of advanced materials, chemicals, and fibers f

In [26]:
res

,Ticker,Company Name,Sector,Market Cap,Summary
0,TESTER,N/A,N/A,N/A,I lack the information to answer. I need the c...
1,MMM,3M Company,Industrials,$82.8B,3M is a multinational conglomerate that manufa...
2,AOS,A. O. Smith Corporation,Industrials,$9.5B,A. O. Smith is a global manufacturer of reside...
3,ABT,Abbott Laboratories,Healthcare,$229.1B,Abbott Laboratories is a global healthcare com...
4,ABBV,AbbVie Inc.,Healthcare,$338.8B,AbbVie is a global biopharmaceutical company t...
...,...,...,...,...,...
382,PRU,"Prudential Financial, Inc.",Financial Services,$40.4B,"Prudential Financial, Inc. is a global financi..."
383,PEG,Public Service Enterprise Group Incorporated,Utilities,$41.9B,Public Service Enterprise Group (PSEG) is a di...
384,PSA,Public Storage,Real Estate,$52.9B,"Public Storage is the world's largest owner, o..."
385,PHM,"PulteGroup, Inc.",Consumer Cyclical,$22.8B,"PulteGroup, Inc. is a homebuilder in the Unite..."


In [7]:
res.to_csv(list_name+'_annotation.csv', index=False)

In [ ]:
# list_name='ftse250'


# # ending=''
# filel=open('../gathering/data/'+list_name+'.txt','a+')
# filel.seek(0)
# stocklist=filel.read().splitlines()
# filel.close()

In [ ]:
res=stock_list_annotation(stocklist,client,'UK')

In [ ]:

# def stock_list_annotation(tickers, client,nation):
#     """Get top N intraday losers from a predefined S&P 500 ticker list with company details
    
#     Returns DataFrame with company name, sector, market cap, current price, and percentage change.
#     Includes random sleep intervals between API calls to prevent rate limiting.
#     """
#     losers = []
#     ending=nation_ending[nation]

    
#     for ticker in tickers:
#         print(ticker)
#         try:
#             #Random sleep between 0 and 3 seconds
            


#             stock = yf.Ticker(ticker+ending)
#             data = stock.history(period='1d', interval='1m')
#             dic={
#                 'Ticker': ticker,
#                 'Company Name': stock.info.get('longName', 'N/A'),
#                 'Sector': stock.info.get('sector', 'N/A'),
#                 'Market Cap': get_market_cap_formatted(stock.info.get('marketCap', None)),
#             }

#             dic['Summary']=summary(dic['Company Name'],nation,client)
#             print(dic['Summary'])
            
#         except Exception as e:
#             dic=dict()
#         losers.append(dic)
#         sleep_time=np.random.gamma(4,5)
#         print(sleep_time)
#         time.sleep(sleep_time)
#     losers_df = pd.DataFrame(losers)
#     if len(losers_df) == 0:
#         return pd.DataFrame(columns=['Ticker', 'Company Name', 'Sector', 'Market Cap', 'Summary'])
    
#     #losers_df = losers_df.sort_values(by='% Change')
#     return losers_df[['Ticker', 'Company Name', 'Sector', 'Market Cap', 'Summary']].reset_index(drop=True)


In [ ]:
# annotated_list=pd.read_csv('data/sp500_annotation.csv')
# annotated_list=annotated_list.set_index('Symbol')
# annotated_list.to_csv('data/sp500_annotation.csv')
# list_name='ftse100'

# list_name='SP500'
# # ending=''
# filel=open('../gathering/data/'+list_name+'.txt','a+')
# filel.seek(0)
# stocklist=filel.read().splitlines()
# filel.close()